In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as dd
import dask.array as da
import dask.bag as db

In [2]:
def product_lookup(itemNumbers):
    
    if isinstance(itemNumbers, int):
        itemNumbers = [itemNumbers]
        
    db = product_info[product_info.ItemNumber.isin(itemNumbers)]
        
    
    if db.shape[0] == 0:
        return "No item found"
    
    return db

def revenue_stitcher(invoices, itemNumbers, all_time = False):
    
    items = product_lookup(itemNumbers)

    if all_time:
        sales = invoices[invoices.ItemNumber.isin(itemNumbers)]

        new_df = pd.merge(items, sales)

        del items
        del sales

        new_df['ProfitPerBottle'] = new_df.StateBottleRetail - new_df.StateBottleCost
        new_df['ProfitPerCase'] = new_df.StateBottleRetail * new_df.Pack - new_df.StateCaseCost

        new_df['Overall_Revenue'] = new_df.BottlesSold * new_df.StateBottleRetail
        new_df['Overall_Profit'] = new_df.ProfitPerBottle * new_df.BottlesSold

        return new_df
    
    else:
        sales = invoices[invoices.ItemNumber.isin(itemNumbers)]

        sales = sales.pivot(index='ItemNumber', columns = 'Year', values = ['BottlesSold', 'UniqueTransactions']).fillna(0)

        sales.columns = [x[0]+"_"+x[1] for x in sales_pivot.columns.to_flat_index()]

        new_df = pd.merge(items, sales, left_on = 'ItemNumber', right_index = True)
        
        #tbd
        

In [3]:
alltime_sales = pd.read_csv('./data/alltime_sales.csv')
yearly_sales = pd.read_csv('./data/yearly_sales.csv')

In [5]:
product_info = pd.read_csv('./data/Iowa_Liquor_Products.csv')